In [1]:
import os

#os.environ["HF_DATASETS_CACHE"] = "/domino/datasets/local/SAS-R/"
os.environ["TRANSFORMERS_CACHE"] =  "/mnt/data/CodeGen"

In [2]:
from transformers.utils import TRANSFORMERS_CACHE
TRANSFORMERS_CACHE

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'/mnt/data/CodeGen'

In [3]:
import torch

print(torch.cuda.is_available())
print(torch.__version__)

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import transformers

True
2.0.1+cu118


In [4]:
model_name = 'codellama/CodeLlama-7b-Instruct-hf'
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=TRANSFORMERS_CACHE)
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             cache_dir=TRANSFORMERS_CACHE,
                                             torch_dtype=torch.bfloat16,
                                             device_map="auto",
                                            )


Loading checkpoint shards: 100%|██████████| 2/2 [00:13<00:00,  6.62s/it]


In [6]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer
               )

In [7]:
code="""
/* Load the input dataset */
proc import out=cars_data
    datafile="/mnt/data/mtcars.csv"
    dbms=csv
    replace;
    getnames=YES;
run;

/* Sort the input dataset by displacement */
proc sort data=cars_data;
      by disp;
run;

/* Perform the linear regression */
proc reg data=cars_data;
      model mpg = disp / noprint;
      output out=output_data predicted=mpg_predicted;
run;

/* Print the regression results */
/* In this case a scatter plot with regression line best fit */
proc sgplot data=output_data;
      scatter x=disp y=mpg / markerattrs=(symbol=circlefilled) name='scatter';
      series x=disp y=mpg_predicted / lineattrs=(color=blue) name='regression';
      keylegend 'scatter' 'regression';
run;
"""

In [42]:
convert="You are a code assistant.Please write  R code that will reproduce the results of the SAS code that follows \n"
thisPrompt="{}\n{}".format(convert, code)

In [43]:
sequences = pipe(
    thisPrompt,
    do_sample=True,
    num_return_sequences=5,
    eos_token_id=tokenizer.eos_token_id,
    max_new_tokens=400
)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [44]:
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Result: You are a code assistant.Please write  R code that will reproduce the results of the SAS code that follows 


/* Load the input dataset */
proc import out=cars_data
    datafile="/mnt/data/mtcars.csv"
    dbms=csv
    replace;
    getnames=YES;
run;

/* Sort the input dataset by displacement */
proc sort data=cars_data;
      by disp;
run;

/* Perform the linear regression */
proc reg data=cars_data;
      model mpg = disp / noprint;
      output out=output_data predicted=mpg_predicted;
run;

/* Print the regression results */
/* In this case a scatter plot with regression line best fit */
proc sgplot data=output_data;
      scatter x=disp y=mpg / markerattrs=(symbol=circlefilled) name='scatter';
      series x=disp y=mpg_predicted / lineattrs=(color=blue) name='regression';
      keylegend 'scatter' 'regression';
run;


Result: You are a code assistant.Please write  R code that will reproduce the results of the SAS code that follows 


/* Load the input dataset */
proc import 